<a href="https://colab.research.google.com/github/mightyoctopus/lora-fine-tuning-example-code/blob/main/LoRA_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install -q peft transformers datasets torch accelerate

In [59]:
!pip install -U bitsandbytes

Dataset: Rabe3/QA_Synthatic_Medical_data

Model (Instrcut): Qwen/Qwen3-0.6B

- quantization
- base model load and load peft model (with LoraConfig)
- tokenization (batches)
- load dataset
- map dataset to be tokenized
- Train (with TrainingArguments configued)


In [60]:
from torch.utils.data import DataLoader
import torch
from datasets import load_dataset, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
                          )
from peft import LoraConfig, get_peft_model, TaskType

from google.colab import drive, userdata
from huggingface_hub import login, snapshot_download

import os

In [61]:
model_name = "Qwen/Qwen3-0.6B"
medical_dataset_name = "Rabe3/QA_Synthatic_Medical_data"
general_dataset_name = "tatsu-lab/alpaca"

In [62]:
drive.mount("/content/drive")
cache_path = "/content/drive/My Drive/models/huggingface_cache"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model_folder_name = "models--" + model_name.replace("/", "--")
parent_model_path_in_drive = os.path.join(cache_path, model_folder_name)

if not os.path.exists(parent_model_path_in_drive):
    print("Model not found in Drive -- downloading from HF...")
    model_path = snapshot_download(
        repo_id=model_name,
        cache_dir=cache_path,
        local_dir_use_symlinks=False
    )
else:
    print("Model found in Drive -- fetching from the cache...")
    snapshots_dir = os.path.join(parent_model_path_in_drive, "snapshots")
    drive_id = os.listdir(snapshots_dir)

    if drive_id:
        model_path = os.path.join(snapshots_dir, drive_id[0])
    else:
        raise ValueError("No snapshot found in the cache path.")

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    cache_dir=cache_path
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


Model found in Drive -- fetching from the cache...


In [64]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [65]:
data_medical = load_dataset(medical_dataset_name, "default", split="train[:2000]")
### Add general dataset as extra for better generalization of the model
general_dataset =load_dataset(general_dataset_name, split="train")


In [66]:
# Take 10–20% of general dataset for balance
general_ratio = 0.15
g_count = round(len(data_medical) * general_ratio / (1 - general_ratio))

# Shuffle and sample
data_general = general_dataset.shuffle(seed=42).select(range(min(g_count, len(general_dataset))))

# Merge both
data_mixed = concatenate_datasets([data_medical, data_general]).shuffle(seed=42)

print(f"Medical: {len(data_medical)}, General: {len(data_general)}, Mixed: {len(data_mixed)}")


Medical: 2000, General: 353, Mixed: 2353


In [67]:
def tokenize(batch):
    texts = []

    for convo in batch["conversations"]:
        for turn in convo:
            human_msg = turn["value"] if turn["from"] == "human" else ""
            assisant_msg = turn["value"] if turn["from"] == "gpt" else ""

            texts.append(f"### Instruction:\n{human_msg}\n### Response:\n{assisant_msg}")


    tokens = tokenizer(
        texts,
        padding="max_length",
        max_length=256,
        truncation=True,
        return_tensors="pt"
    )

    print("TOKENS", tokens)

    tokens["labels"] = tokens["input_ids"].clone()

    return tokens

In [70]:
tokenized_data = data_medical.map(tokenize, batched=True, remove_columns=['conversations', 'source', 'score', 'metadata'])

In [71]:
training_args = TrainingArguments(
    output_dir = "./fine_tuned_result",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate= 5e-5,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="none",
    save_total_limit=1
)

In [72]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    processing_class=tokenizer
)

In [73]:
result = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Step,Training Loss
10,9.572400
20,8.960300
30,7.334500
40,6.167600
50,4.569700
60,2.955900
70,1.928200
80,1.603400
90,1.488500
100,1.420000


In [74]:
print(result.training_loss)

2.641915946960449


In [75]:
### Get Perplexity:
import math

loss = result.training_loss
perplexity = math.exp(loss)

print(f"Perplexity: {perplexity}")

Perplexity: 14.040077899229596


In [76]:
### Log in to Hugging Face to push the model
user_token = userdata.get("HF_TOKEN")
login(user_token)


model.push_to_hub("MightyOctopus/qwen3-0.6B-lora-medical")
tokenizer.push_to_hub("MightyOctopus/qwen3-0.6B-lora-medical")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  12%|#2        |  567kB / 4.60MB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpybyr6iea/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/MightyOctopus/qwen3-0.6B-lora-medical/commit/99cb094f240baf8c9d3773a9d9e8bd06d8571b4f', commit_message='Upload tokenizer', commit_description='', oid='99cb094f240baf8c9d3773a9d9e8bd06d8571b4f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MightyOctopus/qwen3-0.6B-lora-medical', endpoint='https://huggingface.co', repo_type='model', repo_id='MightyOctopus/qwen3-0.6B-lora-medical'), pr_revision=None, pr_num=None)